In [5]:
import os
from dotenv import load_dotenv

# Load the environment variables from the .env file
load_dotenv()

# Get the value of OPENAI_API_KEY from the environment variables
openai_api_key = os.getenv("OPENAI_API_KEY")

In [1]:
import json

# Step 1: Create a basic Python dictionary with the entries "name", "order_id", "status"
orders = {
    "order_12345": {
        "name": "John Doe",
        "order_id": "order_12345",
        "status": "In transit"
    },
    "order_67890": {
        "name": "Jane Smith",
        "order_id": "order_67890",
        "status": "Delivered"
    },
    "order_24680": {
        "name": "Alice Johnson",
        "order_id": "order_24680",
        "status": "Processing"
    }
}

# This is the function that we want the model to be able to call
def get_order_status(order_id: str) -> dict:
    # Lookup the order in the dictionary
    order = orders.get(order_id)
    if order:
        return {"order_id": order_id, "status": order["status"]}
    else:
        return {"order_id": order_id, "status": "Order not found"}

In [2]:
# Step 2: Describe your function to the model so it knows how to call it
function_definition = {
    "name": "get_order_status",
    "description": "Get the status of a customer's order. Call this whenever you need to know the status of an order.",
    "parameters": {
        "type": "object",
        "properties": {
            "order_id": {
                "type": "string",
                "description": "The customer's order ID.",
            },
        },
        "required": ["order_id"],
        "additionalProperties": False,
    }
}

In [3]:
# Step 3: Pass your function definitions as available “tools” to the model, along with the messages
tools = [
    {
        "type": "function",
        "function": function_definition
    }
]

In [65]:
messages = [
    {"role": "system", "content": "You are a helpful customer support assistant. Use the supplied tools to assist the user."},
    {"role": "user", "content": "Hi, can you tell me the status of my order?"}
]

In [66]:
import openai
from openai import OpenAI
openai.api_key = openai_api_key

client = OpenAI()

response = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
)

In [68]:
print(response.choices[0].message)

ChatCompletionMessage(content='Sure, I can help with that. Please provide me with your order ID, so I can check the status for you.', refusal=None, role='assistant', function_call=None, tool_calls=None)


In [64]:
tool_call = response.choices[0].message.tool_calls[0]
print(tool_call)


ChatCompletionMessageToolCall(id='call_0wwfmeKfEregpEKtjm6zpUUX', function=Function(arguments='{\n  "order_id": "order_12345"\n}', name='get_order_status'), type='function')


In [63]:
arguments = json.loads(tool_call.function.arguments)  # Correct way to access the arguments

order_id = arguments.get('order_id')

# Call the get_delivery_date function with the extracted order_id
delivery_date = get_order_status(order_id)
print(delivery_date)

{'order_id': 'order_12345', 'status': 'In transit'}


In [73]:
from pydantic import BaseModel
from openai import OpenAI

client = OpenAI()

class CalendarEvent(BaseModel):
    name: str
    date: str
    participants: list[str]

completion = client.beta.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    messages=[
        {"role": "system", "content": "Extract the event information."},
        {"role": "user", "content": "Alice and Bob are going to a science fair on Friday."},
    ],
    response_format=CalendarEvent,
)

event = completion.choices[0].message.parsed

name='Vergiftung' date='2023-10-04T18:00:00' participants=['Ich', 'Thomas Schmidt']


In [74]:
from pydantic import BaseModel
from openai import OpenAI

client = OpenAI()

class Step(BaseModel):
    explanation: str
    output: str

class MathReasoning(BaseModel):
    steps: list[Step]
    final_answer: str

completion = client.beta.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    messages=[
        {"role": "system", "content": "You are a helpful math tutor. Guide the user through the solution step by step."},
        {"role": "user", "content": "how can I solve 8x + 7 = -23"}
    ],
    response_format=MathReasoning,
)

math_reasoning = completion.choices[0].message

In [75]:
print(math_reasoning)

ParsedChatCompletionMessage[MathReasoning](content='{"steps":[{"explanation":"We want to isolate the term with the variable \'x\'. To start, we need to get rid of the constant term on the left-hand side. To do this, subtract 7 from both sides of the equation.","output":"8x + 7 - 7 = -23 - 7"},{"explanation":"Simplifying both sides after subtracting 7 from each, the 7s cancel on the left, leaving just the term with \'x\'. Calculate the right side as well.","output":"8x = -30"},{"explanation":"Now we need to solve for \'x\'. Since \'x\' is being multiplied by 8, divide each side of the equation by 8 to isolate \'x\'.","output":"8x / 8 = -30 / 8"},{"explanation":"Divide both sides by 8 to solve for \'x\'.","output":"x = -30 / 8"},{"explanation":"Simplify the fraction to find the most reduced form of the answer. Both -30 and 8 can be divided by 2.","output":"x = -15 / 4"}],"final_answer":"x = -15/4"}', refusal=None, role='assistant', function_call=None, tool_calls=[], parsed=MathReasoning(

# Praxis

In [80]:
from pydantic import BaseModel
from openai import OpenAI

client = OpenAI()

user_input = """
Lieferantenrechnung
Rechnungsnummer: 12347
Datum: 17.08.2024
Lieferant: Textil GmbH
Adresse: Musterstraße 1, 12345 Beispielstadt
Bestellnummer: PO-2024-003
Artikelnummer Beschreibung Menge Preis (EUR)
1007 Seidenstoff, 30m Rolle 5 800.00
1008 Metallknöpfe, 50 Stück 100 1.20
1009 Spitzenborte, 2m 200 2.50
Gesamtbetrag (EUR): 1600.00
"""

class ResearchPaperExtraction(BaseModel):
    order_number: str
    article_number: list[str]
    description: list[str]
    amount: list[str]
    prices: list[str]
    total: str
    date: int

completion = client.beta.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    messages=[
        {"role": "system", "content": "You are an expert at structured data extraction. You will be given unstructured text from a order and should convert it into the given structure."},
        {"role": "user", "content": user_input}
    ],
    response_format=ResearchPaperExtraction,
)

research_paper = completion.choices[0].message

In [79]:
print(research_paper)

order_number='PO-2024-003' article_number=['1007', '1008', '1009'] description=['Seidenstoff, 30m Rolle', 'Metallknöpfe, 50 Stück', 'Spitzenborte, 2m'] amount=['5', '100', '200'] prices=['800.00', '1.20', '2.50'] total='1600.00' date=20240817
